In [27]:
%pylab



Using matplotlib backend: MacOSX
Populating the interactive namespace from numpy and matplotlib


In [28]:
class pool:
    def __init__(self, feeTier, initPrice):
        self.feeTier = feeTier
        self.price = initPrice
        self.liquidity = 0*linspace(-100, 100, 201)     
    def addLiquidity(self, amount, lowerTick, upperTick):
        liquidity = amount / (upperTick - lowerTick)
        assert liquidity > 0, "Liquidity is 0"
        self.liquidity[lowerTick] = amount / 2
        self.liquidity[upperTick] = amount / 2
        #self.liquidity[lowerTick:upperTick] += liquidity
    def removeLiquidity(self, amount, lowerTick, upperTick):
        liquidity = amount / (upperTick - lowerTick)        
        assert min(self.liquidity) >= liquidity, 'Cannot remote liquidity'
        self.liquidity[lowerTick:upperTick] -= liquidity
    def buyOption(self, amount, strike, width):
        if min(self.liquidity[(strike-int(floor(width/2))):(strike+int(ceil(width/2)))]) > amount/width:
            self.liquidity[(strike-int(floor(width/2))):(strike+int(ceil(width/2)))] -= amount/width           
        else:
            print('noliquidity')
        if strike >= self.price:
            self.liquidity[-1] += amount
        elif strike < self.price:
            self.liquidity[0] += amount      
                
    def sellOption(self, amount, strike, width):
        liquidity = amount / 1
        removedAmount = 0
        if strike <= self.price:
            if self.liquidity[0] > amount:
                removedAmount += min(amount, self.liquidity[0])
                self.liquidity[0] -= min(amount, self.liquidity[0])       
                self.liquidity[(strike-int(floor(width/2))):(strike+int(ceil(width/2)))] += removedAmount/width
            else:
                print('No liquidity at MIN_TICK')
        elif strike >= self.price:
            if self.liquidity[-1] > amount:
                removedAmount += min(amount, self.liquidity[-1])
                self.liquidity[-1] -= min(amount, self.liquidity[-1])         
                self.liquidity[(strike-int(floor(width/2))):(strike+int(ceil(width/2)))] += removedAmount/width     
            else:
                print('No liquidity at MAX_TICK')                
    def updatePrice(self, newPrice):
        self.price = newPrice

In [66]:
##  Create pool
optionPool = pool(0.01, 100)

## deposit 10 ETH to all ticks
optionPool.addLiquidity(2010, 0, 200)

## Number of steps
N = 250

## Sell Random Options
for i in range(100):
    optAmount = np.random.exponential(25)  # Exponentially distributed option size
    strike = int(100*exp((np.random.normal(0, 0.15, 1)[0])))  # lognormally distributed strike around 100
    optWidth = np.random.randint(3, 5)**2  # Width of the option (9, 16)
    buy = np.random.rand() > 1  # Buy criteria (100% buy)    
    
    if buy:
        optionPool.buyOption(optAmount, strike, optWidth)  # Buy the option
        ## Descriptive text
        text(strike, 83, 'Buy '+str(optAmount)+' at '+str(strike)+', width = '+str(optWidth), fontsize=4, color='#eb6841')
        fill_between([strike - optWidth, strike + optWidth], [80, 80], color='#eb6841', alpha=0.25)        
    else:
        optionPool.sellOption(optAmount, strike, optWidth)  # Sell the option
        ## Descriptive text
        text(strike, 83, 'Sell '+str(optAmount)+' at '+str(strike)+', width = '+str(optWidth), fontsize=4, color='#35aaaf')  
        fill_between([strike - optWidth, strike + optWidth], [80, 80], color='#35aaaf', alpha=0.25)    
        
    step(linspace(0, 201, 201), optionPool.liquidity)  # plot the liquidity distribution 
    title('Only selling is allowed')
    ylim((0, 100))    
    plt.draw()
    #plt.savefig('/Users/guillaume/Temp/yewbow/Animations/test-'+f'-{i:04}'+'.png', dpi=900)         
    plt.pause(0.00001)
    plt.clf()  
    
## Simulate a GBM price

simPrice = 100*(exp(cumsum(vstack((zeros((1, 1)), 0.05*(np.random.normal(0, np.sqrt(0.1), size=(1, N)).T))), 0)))

## Simulate an exponentially distributed waiting times
timeArray = (cumsum(random.exponential(1, N)))

## Initialize buy/sell array
buys = []
sells = []

for i in range(N):
    tt = int(timeArray[i])  # Update to current time
    optionPool.updatePrice(simPrice[tt])  # Update the price
    optAmount = int(1000*np.random.exponential(5))/1000  # Exponentially distributed option size
    strike = int(optionPool.price*exp((np.random.normal(0, 0.1, 1)[0])))  # lognormally distributed strike around current price
    optWidth = np.random.randint(1, 4)**2  # Width of the option (1, 4, 9)
    buy = np.random.rand() >= 0.5  # Buy condition (50% buy orders)    

    if buy:
        optionPool.buyOption(optAmount, strike, optWidth)  # Buy the option
        buys.append([tt, -optAmount, strike, optWidth])  # Append buy order to buys array
        ## Descriptive text        
        text(strike, 83, 'Buy '+str(optAmount)+' at '+str(strike)+', width = '+str(optWidth), fontsize=4, color='#eb6841')
        fill_between([strike - optWidth, strike + optWidth], [80, 80], color='#eb6841', alpha=0.25)
    else:
        optionPool.sellOption(optAmount, strike, optWidth)  # Buy the option
        sells.append([tt, optAmount, strike, optWidth])  # Append buy order to buys array       
        ## Descriptive text        
        text(strike, 83, 'Sell '+str(optAmount)+' at '+str(strike)+', width = '+str(optWidth), fontsize=4, color='#35aaaf')  
        fill_between([strike - optWidth, strike + optWidth], [80, 80], color='#35aaaf', alpha=0.25)
        
    step(linspace(0, 201, 201), optionPool.liquidity, color='#231f20') # plot the liquidity distribution 
    plot([optionPool.price, optionPool.price], [0, 200], color='#cc333f')  # plot the current price
    title('delta = '+str(sum(optionPool.liquidity[linspace(0, 201, 201) < optionPool.price])/sum(optionPool.liquidity)))
    plt.draw()
    ylim((0, 100))
    #plt.savefig('/Users/guillaume/Temp/yewbow/Animations/test-'+f'-{i:04}'+'.png', dpi=900)         
    plt.pause(0.1)

    plt.clf()      

No liquidity at MAX_TICK
No liquidity at MAX_TICK
No liquidity at MAX_TICK
No liquidity at MAX_TICK
No liquidity at MAX_TICK
No liquidity at MAX_TICK
No liquidity at MAX_TICK
No liquidity at MAX_TICK
No liquidity at MAX_TICK
No liquidity at MAX_TICK
No liquidity at MAX_TICK
No liquidity at MAX_TICK
No liquidity at MAX_TICK
No liquidity at MAX_TICK
No liquidity at MAX_TICK
No liquidity at MAX_TICK
No liquidity at MAX_TICK
No liquidity at MAX_TICK
noliquidity
No liquidity at MAX_TICK
No liquidity at MAX_TICK
No liquidity at MAX_TICK
noliquidity
noliquidity
noliquidity
noliquidity
noliquidity
noliquidity
noliquidity
noliquidity
noliquidity
noliquidity
noliquidity
noliquidity
noliquidity
noliquidity
noliquidity
noliquidity


IndexError: index 256 is out of bounds for axis 0 with size 251